# 한글설정

In [1]:
!apt-get install fonts-nanum* > /dev/null 2>&1
!fc-cache -fv > /dev/null 2>&1

In [2]:
import os
import matplotlib as mpl

file_dir = os.path.split(mpl.__file__)[0]
font_dir = os.path.join(file_dir, 'mpl-data/fonts/ttf')
!cp /usr/share/fonts/truetype/nanum/Nanum* {font_dir}
!rm -rf ~/.cache/matplotlib/*

> Runtime Restart

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='NanumGothicCoding')

# 데이터 가져오기

In [2]:
import os
import gdown

id = "10Hpa4YM0KX_Ig0W9w7DbTdq62nF2UThA"
output = "./open.zip"

if not os.path.isdir('./datasets'):
  gdown.download(id=id, output=output)
  gdown.extractall(path=output, to='./datasets')

# 데이터프레임 만들기

In [1]:
import os
import glob
import pandas as pd
from collections import defaultdict

raw_data = defaultdict(pd.DataFrame)

for fname in glob.glob('./**/*.csv', recursive=True):
    df_name = os.path.splitext(os.path.basename(fname))[0]
    raw_data[df_name] = pd.read_csv(fname)

# Weights & Biases 설정

In [9]:
!pip install pathlib ruamel-yaml

In [10]:
!pip install wandb -Uq

In [118]:
result_ex1 = pd.read_csv('./datasets/Feature_Except_1.csv')
base_nrmse = result_ex1[result_ex1['remove_column'] == 'All']['LG_NRMSE_MEAN'].values[0]
ex_mean = result_ex1[result_ex1['LG_NRMSE_MEAN'] < base_nrmse]['remove_column'].values.tolist()
required_column = train.filter(regex='X').drop(columns=ex_mean).columns.to_list()

32

In [131]:
result_ex2 = pd.read_csv('./datasets/Feature_Except_2.csv')
result_ex2.head()


,Name,State,Notes,User,Tags,Created,Runtime,Sweep,add_column,LG_NRMSE_BEST,...,Y_05_NRMSE,Y_06_NRMSE,Y_07_NRMSE,Y_08_NRMSE,Y_09_NRMSE,Y_10_NRMSE,Y_11_NRMSE,Y_12_NRMSE,Y_13_NRMSE,Y_14_NRMSE
0,quiet-sweep-12,finished,-,zbooster,NaN,2022-08-16T03:15:13.000Z,114,1537riwk,X_14,1.934971,...,0.079127,0.103242,0.128978,0.023879,0.023693,0.038067,0.033196,0.023831,0.023782,0.023816
1,major-sweep-30,finished,-,zbooster,NaN,2022-08-16T03:51:09.000Z,125,1537riwk,X_30,1.935857,...,0.079175,0.103402,0.129010,0.023799,0.023608,0.038084,0.033159,0.023748,0.023702,0.023742
2,vital-sweep-27,finished,-,zbooster,NaN,2022-08-16T03:44:58.000Z,120,1537riwk,X_32,1.936377,...,0.079217,0.103455,0.129039,0.023796,0.023610,0.038084,0.033169,0.023748,0.023699,0.023739
3,legendary-sweep-26,finished,-,zbooster,NaN,2022-08-16T03:43:02.000Z,114,1537riwk,X_15,1.935445,...,0.079173,0.103198,0.129052,0.023893,0.023704,0.038096,0.033207,0.023842,0.023794,0.023831
4,smooth-sweep-15,finished,-,zbooster,NaN,2022-08-16T03:21:25.000Z,114,1537riwk,X_18,1.936237,...,0.079157,0.103532,0.129067,0.023890,0.023702,0.038095,0.033201,0.023841,0.023791,0.023826


In [222]:
required_column

['X_03',
 'X_04',
 'X_05',
 'X_06',
 'X_07',
 'X_08',
 'X_09',
 'X_10',
 'X_11',
 'X_16',
 'X_17',
 'X_19',
 'X_20',
 'X_22',
 'X_26',
 'X_28',
 'X_29',
 'X_31',
 'X_33',
 'X_38',
 'X_42',
 'X_44',
 'X_45',
 'X_49']

In [218]:

base_nrmse = result_ex2[result_ex2['Sweep'] == '3vy4v14g']['LG_NRMSE_MEAN'].values[0]
all_add = result_ex2[result_ex2['LG_NRMSE_MEAN'] < base_nrmse]['add_column'].values.tolist()



In [219]:
for cname in ['X_13', 'X_47', 'X_23', 'X_48', 'X_30', 'X_41', 'X_43']:
     all_add.remove(cname)

In [220]:
all_add

['X_14', 'X_32', 'X_15', 'X_18', 'X_46', 'X_01']

In [227]:
required_column.extend(all_add)
len(set(required_column))

30

In [209]:
from itertools import combinations

# len(list(combinations(all_add, len(all_add)-1)))
ex1 = []
for val in combinations(all_add, len(all_add)-1):
    ex1.append(list(val))

len(ex1), ex1[:1]


(6, [['X_14', 'X_32', 'X_15', 'X_18', 'X_46']])

In [210]:
import wandb

wandb.login()

True

In [211]:
sweep_config = {
    'method': 'grid'
}

metric = {
    'name': 'LG_NRMSE'
}

sweep_config['metric'] = metric

parameters_dict = {
    'add_column': {
        # 'values': ex_mean
        'values': ex1
    },
}

sweep_config['parameters'] = parameters_dict

In [212]:
sweep_id = wandb.sweep(sweep_config, project="Antenna_Feature_Remove-1")

Create sweep with ID: czzdzef0
Sweep URL: https://wandb.ai/zbooster/Antenna_Feature_Remove-1/sweeps/czzdzef0


# 평가함수

In [213]:
import numpy as np
from sklearn import metrics

# 데이콘에서 제공한 평가함수
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    # gt = gt.to_numpy()
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score, all_nrmse

# 학습하기

In [214]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer

scaler_list = [
    ('Unscaled', None),
    ('Standard', StandardScaler()),
    ('Min-Max', MinMaxScaler()),
    ('Min-Abs', MaxAbsScaler()),
    ('Robust', RobustScaler(quantile_range=(25, 75))),
    ('uniform pdf', QuantileTransformer(output_distribution="uniform")),
    ('gaussian pdf', QuantileTransformer(output_distribution="normal")),
    ('L2 normalizing', Normalizer())
]

In [215]:
import time
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from collections import defaultdict

train = raw_data['train'].copy()

def run_train(config=None):
  with wandb.init(config=config):

    config = wandb.config

    kf = KFold(shuffle=True, random_state=13)
    result = defaultdict(list)

    # 데이터 나누기
    tmpC = required_column.copy()
    tmpC.extend(config.add_column)
    X = train.filter(items=tmpC).values
    # X = train.filter(regex='X').drop(columns=config.remove_column, errors='ignore').values
    # X = train.filter(regex='X').values
    y = train.filter(regex='Y').values
    for idx, (train_index, test_index) in enumerate(kf.split(X)):

      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

      # 파이프라인 작성
      pipe = make_pipeline( 
        RandomForestRegressor(criterion="squared_error", random_state=13, n_jobs=-1
                            , n_estimators=200
                            , max_depth=50
                            , min_samples_leaf=4
                            , min_samples_split=10)
      )

      # 학습하기
      pipe.fit(X_train, y_train)

      # 검증하기
      y_pred = pipe.predict(X_test)

      # 평가 및 기록
      score, all_nrmse = lg_nrmse(y_test, y_pred)
      result['LG_NRMSE'].append(score)
      for i, v in enumerate(all_nrmse):
        result['Y_%02d_NRMSE' % (i+1)].append(v)
    
    log_dict = defaultdict(float)
    log_dict['LG_NRMSE_MEAN'] = np.mean(result['LG_NRMSE'])
    log_dict['LG_NRMSE_BEST'] = min(result['LG_NRMSE'])
    for i in range(14):
      log_dict['Y_%02d_NRMSE' % (i+1)] = np.mean(result['Y_%02d_NRMSE' % (i+1)])
    wandb.log(log_dict)

In [216]:
wandb.agent(sweep_id, run_train)

wandb: Agent Starting Run: gj2cdhee with config:
wandb: 	add_column: ['X_14', 'X_32', 'X_15', 'X_18', 'X_46']
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


LG_NRMSE_BEST,▁
LG_NRMSE_MEAN,▁
Y_01_NRMSE,▁
Y_02_NRMSE,▁
Y_03_NRMSE,▁
Y_04_NRMSE,▁
Y_05_NRMSE,▁
Y_06_NRMSE,▁
Y_07_NRMSE,▁
Y_08_NRMSE,▁
Y_09_NRMSE,▁


wandb: Agent Starting Run: wcy68uvn with config:
wandb: 	add_column: ['X_14', 'X_32', 'X_15', 'X_18', 'X_01']
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


LG_NRMSE_BEST,▁
LG_NRMSE_MEAN,▁
Y_01_NRMSE,▁
Y_02_NRMSE,▁
Y_03_NRMSE,▁
Y_04_NRMSE,▁
Y_05_NRMSE,▁
Y_06_NRMSE,▁
Y_07_NRMSE,▁
Y_08_NRMSE,▁
Y_09_NRMSE,▁


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 54k06vov with config:
wandb: 	add_column: ['X_14', 'X_32', 'X_15', 'X_46', 'X_01']
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: Agent Starting Run: pnma1hkc with config:
wandb: 	add_column: ['X_14', 'X_15', 'X_18', 'X_46', 'X_01']
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


LG_NRMSE_BEST,▁
LG_NRMSE_MEAN,▁
Y_01_NRMSE,▁
Y_02_NRMSE,▁
Y_03_NRMSE,▁
Y_04_NRMSE,▁
Y_05_NRMSE,▁
Y_06_NRMSE,▁
Y_07_NRMSE,▁
Y_08_NRMSE,▁
Y_09_NRMSE,▁


wandb: Agent Starting Run: qbkcoix3 with config:
wandb: 	add_column: ['X_32', 'X_15', 'X_18', 'X_46', 'X_01']
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


LG_NRMSE_BEST,▁
LG_NRMSE_MEAN,▁
Y_01_NRMSE,▁
Y_02_NRMSE,▁
Y_03_NRMSE,▁
Y_04_NRMSE,▁
Y_05_NRMSE,▁
Y_06_NRMSE,▁
Y_07_NRMSE,▁
Y_08_NRMSE,▁
Y_09_NRMSE,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
